此示例显示如何定义和使用自定义因子。它还显示股价数据和`Fundamentals`如何一起使用。

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2017-1-1 --end 2018-4-25
from logbook import Logger
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

log = Logger('回测')

from zipline.pipeline import CustomFactor
from zipline.pipeline.fundamentals import Fundamentals
from zipline.pipeline.builtin import quarterly_multiplier, changed_locations, nanmean


# 该类位于`zipline.pipeline.builtin`模块，此为简化版本，用于演示如何自定义因子
class TTMSale(CustomFactor):
    """
    最近一年营业总收入季度平均数

    Notes:
    ------
    trailing 12-month (TTM)
    """
    inputs = [
        Fundamentals.profit_statement.A001,
    ]
    # 一年的交易日不会超过260天
    window_length = 260

    def _validate(self):
        super(TTMSale, self)._validate()
        if self.window_length < 260:
            raise ValueError('window_length值必须大于260,以确保获取一年的财务数据')

    def compute(self, today, assets, out, sales):
        res = nanmean(sales, axis=0)
        # 收入单位为万元
        out[:] = res * 10000

def initialize(context):

    # Note that we don't call add_factor on these Factors.
    # We don't need to store intermediate values if we're not going to use them
    sma_short = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)
    sma_long = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=100)

    sma_quotient = sma_short / sma_long

    # 构造自定义因子
    ttm_sale = TTMSale()

    # 滚动营业总收入前500位过滤器
    ttm_sale_top_500 = ttm_sale.top(500)

    # Construct an average dollar volume factor
    dollar_volume = AverageDollarVolume(window_length=30)

    # Define high dollar-volume filter to be the top 10% of stocks by dollar
    # volume.
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    high_sale_high_dv = ttm_sale_top_500 & high_dollar_volume

    # Create filters for our long and short portfolios.
    longs = sma_quotient.top(200, mask=high_sale_high_dv)
    shorts = sma_quotient.bottom(200, mask=high_sale_high_dv)

    pipe_columns = {
        'longs': longs,
        'shorts': shorts,
        'ttm_sale': ttm_sale,
        'sma_quotient': sma_quotient
    }

    # Use multiple screens to narrow the universe
    pipe_screen = (longs | shorts)

    pipe = Pipeline(columns=pipe_columns, screen=pipe_screen)
    attach_pipeline(pipe, 'example')


def before_trading_start(context, data):
    context.output = pipeline_output('example')
    context.longs = context.output[context.output.longs]
    context.shorts = context.output[context.output.shorts]

    context.security_list = context.shorts.index.union(context.longs.index)

    # Print the 5 securities with the lowest sma_quotient.
    print("空头列表")
    log.info("\n" + str(
        context.shorts.sort_values(['sma_quotient'], ascending=True).head()))

    # Print the 5 securities with the highest sma_quotient.
    print("多头列表")
    log.info("\n" + str(
        context.longs.sort_values(['sma_quotient'], ascending=False).head()))

空头列表
[2018-04-29 21:09:39.200737] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.872838  1.094183e+10
山东黄金(600547)   True    True      0.933966  2.767648e+10
江苏银行(600919)   True    True      0.952294  2.378685e+10
三角轮胎(601163)   True    True      0.955783  4.951380e+09
温氏股份(300498)   True    True      0.962947  3.324144e+10
多头列表
[2018-04-29 21:09:39.208222] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
中国联通(600050)   True    True      1.285196  1.747194e+11
三钢闽光(002110)   True    True      1.281338  7.826532e+09
柳钢股份(601003)   True    True      1.250679  1.583804e+10
中国建筑(601668)   True    True      1.246038  5.581722e+11
驰宏锌锗(600497)   True    True      1.222434  9.522028e+09
空头列表
[2018-04-29 21:09:39.220551] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.874050  1.097421e+10
山东黄金(600547)   True    True      0.933700  2.769687e+10
江苏银行(600919)   True    Tr

空头列表
[2018-04-29 21:09:42.096742] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.879957  1.129808e+10
万 科Ａ(000002)   True    True      0.895446  9.985353e+10
西山煤电(000983)   True    True      0.926637  1.114736e+10
山东黄金(600547)   True    True      0.937836  2.790082e+10
江苏银行(600919)   True    True      0.940678  2.378685e+10
多头列表
[2018-04-29 21:09:42.104837] INFO: 回测: 
               longs  shorts  sma_quotient      ttm_sale
三钢闽光(002110)    True    True      1.277749  7.830426e+09
中国联通(600050)    True    True      1.271379  1.745175e+11
华锦股份(000059)    True    True      1.183245  1.777446e+10
上海物贸(600822)    True    True      1.174425  2.309121e+10
*ST船舶(600150)   True    True      1.152380  1.572853e+10
空头列表
[2018-04-29 21:09:42.124646] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.883833  1.133046e+10
万 科Ａ(000002)   True    True      0.890651  9.999760e+10
西山煤电(000983)   True

空头列表
[2018-04-29 21:09:42.399281] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
万 科Ａ(000002)   True    True      0.870284  9.831651e+10
万达电影(002739)   True    True      0.878913  6.412612e+09
乐视网(300104)    True    True      0.915372  1.152935e+10
欧菲科技(002456)   True    True      0.928920  1.346539e+10
绿地控股(600606)   True    True      0.953479  1.276868e+11
多头列表
[2018-04-29 21:09:42.406930] INFO: 回测: 
               longs  shorts  sma_quotient      ttm_sale
三钢闽光(002110)    True    True      1.242087  7.755441e+09
华锦股份(000059)    True    True      1.213714  1.747041e+10
中国联通(600050)    True    True      1.196253  1.725618e+11
广深铁路(601333)    True    True      1.145299  1.011832e+10
*ST船舶(600150)   True    True      1.140727  1.538009e+10
空头列表
[2018-04-29 21:09:42.418592] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
万 科Ａ(000002)   True    True      0.871697  9.801460e+10
万达电影(002739)   True    True      0.880687  6.414292e+09
乐视网(300104)    True

空头列表
[2018-04-29 21:09:42.608647] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
万 科Ａ(000002)   True    True      0.881410  1.111354e+11
中国长城(000066)   True    True      0.892323  4.331163e+10
杭州银行(600926)   True    True      0.908208  1.373284e+10
欧菲科技(002456)   True    True      0.930890  1.449923e+10
西山煤电(000983)   True    True      0.938027  1.160569e+10
多头列表
[2018-04-29 21:09:42.615450] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
华锦股份(000059)   True    True      1.232820  1.814376e+10
西部建设(002302)   True    True      1.220057  6.659290e+09
三钢闽光(002110)   True    True      1.193791  8.212562e+09
太钢不锈(000825)   True    True      1.172766  3.620909e+10
云南铜业(000878)   True    True      1.146896  3.088189e+10
空头列表
[2018-04-29 21:09:42.628442] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
万 科Ａ(000002)   True    True      0.883882  1.113083e+11
中国长城(000066)   True    True      0.886870  4.329699e+10
杭州银行(600926)   True    Tr

空头列表
[2018-04-29 21:09:42.819571] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
分众传媒(002027)   True    True      0.848506  6.210241e+09
圆通速递(600233)   True    True      0.862904  5.796147e+09
中国长城(000066)   True    True      0.873435  4.315054e+10
中国电影(600977)   True    True      0.891406  6.894144e+09
万 科Ａ(000002)   True    True      0.899867  1.130363e+11
多头列表
[2018-04-29 21:09:42.829228] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
西部建设(002302)   True    True      1.243964  6.711522e+09
华锦股份(000059)   True    True      1.205339  1.806738e+10
三钢闽光(002110)   True    True      1.169215  8.279238e+09
华新水泥(600801)   True    True      1.166969  7.963671e+09
云南铜业(000878)   True    True      1.158440  3.098932e+10
空头列表
[2018-04-29 21:09:42.840756] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
分众传媒(002027)   True    True      0.846579  6.216340e+09
圆通速递(600233)   True    True      0.865624  5.857340e+09
中国长城(000066)   True    Tr

...

空头列表
[2018-04-29 21:09:50.403371] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.702254  8.343792e+09
歌尔股份(002241)   True    True      0.832340  1.283523e+10
深天马Ａ(000050)   True    True      0.860575  7.570926e+09
长信科技(300088)   True    True      0.867048  6.935064e+09
均胜电子(600699)   True    True      0.892870  1.471574e+10
多头列表
[2018-04-29 21:09:50.408114] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
泰禾集团(000732)   True    True      1.498917  1.233411e+10
新城控股(601155)   True    True      1.355662  1.389211e+10
鲁西化工(000830)   True    True      1.314141  8.403440e+09
保利地产(600048)   True    True      1.266480  7.008435e+10
恒力股份(600346)   True    True      1.263975  1.263848e+10
空头列表
[2018-04-29 21:09:50.417271] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.676613  8.282888e+09
歌尔股份(002241)   True    True      0.827297  1.387794e+10
深天马Ａ(000050)   True    Tr

空头列表
[2018-04-29 21:09:50.566341] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.400163  7.673856e+09
歌尔股份(002241)   True    True      0.817928  1.411828e+10
深天马Ａ(000050)   True    True      0.834137  8.190899e+09
领益智造(002600)   True    True      0.859300  8.808337e+09
长信科技(300088)   True    True      0.864818  6.944527e+09
多头列表
[2018-04-29 21:09:50.571450] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
泰禾集团(000732)   True    True      1.408017  1.213341e+10
鲁西化工(000830)   True    True      1.302835  8.408354e+09
新城控股(601155)   True    True      1.296615  1.733657e+10
上海钢联(300226)   True    True      1.294472  4.030498e+10
恒力股份(600346)   True    True      1.232533  1.350436e+10
空头列表
[2018-04-29 21:09:50.582487] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.369401  7.612953e+09
歌尔股份(002241)   True    True      0.818077  1.414232e+10
深天马Ａ(000050)   True    Tr

空头列表
[2018-04-29 21:09:50.735716] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.249538  7.003921e+09
深天马Ａ(000050)   True    True      0.831526  8.329488e+09
歌尔股份(002241)   True    True      0.841457  1.438266e+10
中国铝业(601600)   True    True      0.847959  1.135801e+11
西水股份(600291)   True    True      0.852470  6.281949e+09
多头列表
[2018-04-29 21:09:50.741111] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
上海钢联(300226)   True    True      1.315049  4.167652e+10
泰禾集团(000732)   True    True      1.203510  1.193272e+10
鲁西化工(000830)   True    True      1.192188  8.413267e+09
太钢不锈(000825)   True    True      1.177162  3.874060e+10
新城控股(601155)   True    True      1.173615  1.786780e+10
空头列表
[2018-04-29 21:09:50.748942] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.250140  6.943018e+09
深天马Ａ(000050)   True    True      0.833073  8.342087e+09
中国铝业(601600)   True    Tr

空头列表
[2018-04-29 21:09:50.933839] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.282824  6.333986e+09
中国铝业(601600)   True    True      0.720357  1.151038e+11
新华保险(601336)   True    True      0.838861  9.788162e+10
韶钢松山(000717)   True    True      0.856704  1.537413e+10
歌尔股份(002241)   True    True      0.888740  1.464704e+10
多头列表
[2018-04-29 21:09:50.941334] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
上海钢联(300226)   True    True      1.360082  4.304805e+10
洛阳钼业(603993)   True    True      1.158308  1.463847e+10
易见股份(600093)   True    True      1.155176  9.306062e+09
中国国旅(601888)   True    True      1.139157  1.719492e+10
华东医药(000963)   True    True      1.116922  1.805688e+10
空头列表
[2018-04-29 21:09:50.951492] INFO: 回测: 
              longs  shorts  sma_quotient      ttm_sale
乐视网(300104)    True    True      0.285346  6.273083e+09
中国铝业(601600)   True    True      0.711391  1.152423e+11
新华保险(601336)   True    Tr

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-01-03 07:00:00+00:00,NaN,0.0,NaN,0.009712,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2017-01-04 07:00:00+00:00,0.0,0.0,0.0,0.017593,0.021406,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2017-01-05 07:00:00+00:00,0.0,0.0,0.0,0.017435,0.083084,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2017-01-06 07:00:00+00:00,0.0,0.0,0.0,0.011356,0.115404,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2017-01-09 07:00:00+00:00,0.0,0.0,0.0,0.016261,0.100950,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2017-01-10 07:00:00+00:00,0.0,0.0,0.0,0.014560,0.095760,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2017-01-11 07:00:00+00:00,0.0,0.0,0.0,0.007377,0.104382,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2017-01-12 07:00:00+00:00,0.0,0.0,0.0,0.002279,0.102578,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2017-01-13 07:00:00+00:00,0.0,0.0,0.0,0.002971,0.095975,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2017-01-16 07:00:00+00:00,0.0,0.0,0.0,0.002830,0.090519,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


**时长15秒**